# Google Trends
### Parameters
 - **Search:**
     - Enter term/company of interest
 - **Time_range:**
     - Pick a time range of interest
 - **Related_terms:**
     - <i>Suggested terms:</i> 5 terms that google suggests next when typing
     - <i>Top related queries:</i> the 10 most popular search queries related to your search
     - <i>Rising related queries:</i> 10 queries with the most significant growth in volume in the requested time period. Results with >5000% increase are probably new and had few (if any) prior searches

In [1]:
import pytrends
import pandas as pd 
from pytrends.request import TrendReq
import matplotlib.pyplot as plt 
from datetime import datetime
import time
import seaborn as sns
import matplotlib.dates as mdates
import pytz
from pytz import timezone 

from ipywidgets import interact, interactive, fixed, interact_manual, Button
import ipywidgets as widgets

In [2]:
pytrend = TrendReq(hl = 'en US', tz = 300)
options = ['today 5-y', 'today 12-m','today 1-m', 'today 3-m','now 1-d', 'now 7-d', 'now 1-H', 'now 4-H']
theoptions = ['5 years', '1 year', '1 month','3 months', '1 day', '7 days', '1 hour', '4 hours' ]

In [3]:
def suggestions(keyword):
    df = pd.DataFrame(pytrend.suggestions(keyword))
    d = df[['mid', 'title']]
    new_d = d.set_index('mid')
    return new_d.to_dict()['title']
def related_queries(keyword, spec, correct_format):
    pytrend.build_payload([keyword], cat=0, timeframe=correct_format, geo='', gprop='')    
    related_queries = pytrend.related_queries()
    return related_queries[keyword][spec]

In [4]:
def get_and_plot_suggestions(keyword, wanted_time, param):
    ind = theoptions.index(wanted_time)
    correct_format = options[ind]
    sug = list()
    if param == 'suggested terms':
        sug = [keyword] + list(suggestions(keyword).values())
    elif param == 'top related queries':
        sug = related_queries(keyword, 'top', correct_format)[0:10]['query'].to_list()
    elif param == 'rising related queries':
        rising = related_queries(keyword, 'rising', correct_format)
        percent_inc = rising.set_index('query').to_dict()['value']
        sug = rising[0:10]['query'].to_list()
    looked_at = []
    colors = ['b','g','r','c','m','y','purple','gray','k','b']
    for i in range(len(sug)):
        if sug[i].lower() in looked_at:
            continue
        looked_at.append(sug[i].lower())
        pytrend.build_payload([sug[i]], cat=0, timeframe=correct_format, geo='', gprop='')
        overtime = pd.DataFrame(pytrend.interest_over_time().drop(columns = 'isPartial'))
        
        index = overtime.index
        est = timezone('US/Eastern')
        overtime.index = index.tz_localize(pytz.utc).tz_convert(est)
        
        ax = overtime.plot.line(color = colors[i], figsize = (12,8))
        ax.legend(loc = 'upper right', bbox_to_anchor=(1.0, 1), prop = {'size':15})
        ax.tick_params(axis='both', which='both', labelsize=15)
        ax.set_xlabel('date',fontsize = 15)
        
        if param == 'rising related queries':
            inc = percent_inc[sug[i]]
            ax.set_xlabel('date \n \n {} searches increased {}% in {}'.format(sug[i], inc, wanted_time))
    if param == 'rising related queries':
        rising['% change'] = rising['value']
        rising = rising.drop(columns = 'value')[0:10]
        return rising

In [5]:
@interact_manual 
def run(search = '', related_terms = ['suggested terms', 'top related queries', 'rising related queries'], time_range = theoptions):
    sns.set(color_codes=True)

    return get_and_plot_suggestions(search, time_range, related_terms)


interactive(children=(Text(value='', description='search'), Dropdown(description='related_terms', options=('su…